## Project - Airline AI Assistant

In [149]:
import os
import json
from dotenv import load_dotenv
import google.generativeai as genai
import gradio as gr
from IPython.display import Markdown, display, clear_output

In [150]:
load_dotenv()

google_api_key = os.getenv('GEMINI_API_KEY')

In [151]:
genai.configure(api_key=google_api_key)

In [152]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers using markdown, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [153]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [154]:
price_function = genai.protos.FunctionDeclaration(
    name="get_ticket_price",
    description="Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "destination_city": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                description="The city that the customer wants to travel to"
            )
        },
        required=["destination_city"]
    )
)

In [ ]:
def stream_gemini_with_history(message, history):
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message,
        tools=[price_function]
    )

    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    prompt += f"User: {message}\nAssistant:"

    response = model.generate_content(prompt)

    if response.candidates[0].content.parts and hasattr(response.candidates[0].content.parts[0], "function_call"):
        all_parts = response.candidates[0].content.parts
        function_results = ""

        for part in all_parts:
            if hasattr(part, "function_call"):
                function_call = part.function_call
                function_name = function_call.name

                if function_call.args is not None:
                    function_args = dict(function_call.args)
                else:
                    function_args = {}

                print(f"Function called: {function_name} with args: {function_args}")

                if function_name == "get_ticket_price":
                    dest = function_args.get("destination_city")
                    if dest:
                        result = get_ticket_price(dest)
                        function_results += f"[Function call: {function_name}({function_args}) returned: {result}]\n"

        prompt += "\n" + function_results + "Assistant:"

    stream = model.generate_content(prompt, stream=True)

    response_text = ""
    for chunk in stream:
        if chunk.text:
            response_text += chunk.text
            yield history + [(message, response_text)], ""

In [157]:
history = []
user_message = "hey how are you"

response_gen = stream_gemini_with_history(user_message, history)

for updated_history, _ in response_gen:
    last_user, last_assistant = updated_history[-1]
    clear_output(wait=True)
    display(Markdown(last_assistant))

I am doing well, thank you for asking.
